# 🏥 SimuPaciente UMH - Demo en Google Colab

**Universidad Miguel Hernández de Elche**

Sistema de simulación de pacientes virtuales con voz para práctica de entrevistas clínicas.

---

## 📋 Instrucciones

1. **Ejecuta las celdas en orden** (Shift + Enter)
2. **Proporciona tu API key de OpenAI** cuando se te solicite
3. **Accede a la URL de ngrok** que se generará al final
4. **¡Comienza a practicar!**

⚠️ **IMPORTANTE:** Este notebook usa recursos de Colab gratuitos. La sesión se cerrará automáticamente después de ~12 horas de inactividad.

---

## 🔧 Paso 1: Instalación de Dependencias

Instalamos Python, Node.js y todas las dependencias necesarias.

In [ ]:
%%bash
echo "📦 Instalando dependencias del sistema..."

# Actualizar sistema
apt-get update -qq

# Instalar Node.js 18.x
curl -fsSL https://deb.nodesource.com/setup_18.x | bash -
apt-get install -y nodejs

# Verificar versiones
echo "✅ Node.js version: $(node --version)"
echo "✅ npm version: $(npm --version)"
echo "✅ Python version: $(python3 --version)"

## 📥 Paso 2: Clonar Repositorio

Descargamos el código desde GitHub.

In [ ]:
%%bash
echo "📥 Clonando repositorio desde GitHub..."

# Limpiar si existe
rm -rf /content/ECOE

# Clonar repo
git clone https://github.com/marcosbenghezala/ECOE.git /content/ECOE

echo "✅ Repositorio clonado exitosamente"
ls -la /content/ECOE/

## 🐍 Paso 3: Instalar Dependencias Python

Instalamos las librerías de Python necesarias para el backend.

In [ ]:
%%bash
cd /content/ECOE/simulador

echo "🐍 Instalando dependencias Python..."
pip install -q -r requirements.txt

echo "✅ Dependencias Python instaladas"

## ⚛️ Paso 4: Build del Frontend

Compilamos el frontend React.

In [ ]:
%%bash
cd /content/ECOE/simulador/frontend

echo "⚛️ Instalando dependencias de Node.js..."
npm install --silent

echo "🔨 Building frontend..."
npm run build

echo "✅ Frontend compilado exitosamente"
ls -la /content/ECOE/simulador/frontend/dist/

## 🔑 Paso 5: Configurar API Keys

**IMPORTANTE:** Ingresa tu API key de OpenAI de forma segura.

⚠️ **NUNCA compartas tu API key públicamente**

Obtén tu API key en: https://platform.openai.com/api-keys

In [ ]:
import os
from getpass import getpass

print("🔑 Configuración de API Keys")
print("="*50)

# Solicitar OpenAI API Key de forma segura
openai_key = getpass("Ingresa tu OpenAI API Key: ")

if not openai_key.startswith('sk-'):
    print("⚠️ ADVERTENCIA: La API key no tiene el formato esperado (debe empezar con 'sk-')")
    print("Por favor verifica que hayas copiado la key completa.")
else:
    # Configurar variables de entorno
    os.environ['OPENAI_API_KEY'] = openai_key
    
    # Crear archivo .env
    with open('/content/ECOE/simulador/.env', 'w') as f:
        f.write(f"OPENAI_API_KEY={openai_key}\n")
    
    print("\n✅ API Key configurada exitosamente")
    print(f"✅ Key preview: {openai_key[:8]}...{openai_key[-4:]}")

print("\n📝 Nota: Google Sheets es opcional para esta demo.")
print("   Los resultados se guardarán localmente en la sesión.")

## 🌐 Paso 6: Instalar y Configurar ngrok

ngrok nos permite exponer el servidor local a Internet de forma segura.

In [ ]:
%%bash
echo "🌐 Instalando ngrok..."

# Descargar ngrok
wget -q https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
tar xzf ngrok-v3-stable-linux-amd64.tgz
chmod +x ngrok
mv ngrok /usr/local/bin/

echo "✅ ngrok instalado"
ngrok version

## 🚀 Paso 7: Iniciar Servidor

**¡Última celda!** Esto iniciará el servidor y creará una URL pública.

⚠️ **IMPORTANTE:**
- Esta celda quedará ejecutándose (verás [*] a la izquierda)
- **NO la detengas** mientras uses la aplicación
- Busca la URL de ngrok en la salida (https://xxxx-xx-xxx.ngrok-free.app)
- Copia esa URL y ábrela en una nueva pestaña

In [ ]:
import subprocess
import time
import threading
import requests
from IPython.display import display, HTML

print("🚀 Iniciando SimuPaciente UMH...")
print("="*70)

# Cambiar al directorio del simulador
import os
os.chdir('/content/ECOE/simulador')

# Iniciar ngrok PRIMERO (en background)
print("\n🌐 Iniciando túnel ngrok...")
ngrok_process = subprocess.Popen(
    ['ngrok', 'http', '8080'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)

# Dar tiempo a ngrok para iniciar su API
print("⏳ Esperando a que ngrok esté listo...")
time.sleep(8)

# Iniciar servidor Flask
print("\n🏥 Iniciando servidor Flask...")
server_process = subprocess.Popen(
    ['python3', 'colab_server.py'],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1
)

# Esperar a que el servidor esté listo
print("⏳ Esperando a que el servidor esté listo...")
time.sleep(10)

# Verificar que el servidor está corriendo
server_ok = False
for attempt in range(3):
    try:
        response = requests.get('http://localhost:8080/api/cases', timeout=5)
        print("✅ Servidor Flask iniciado correctamente en puerto 8080")
        server_ok = True
        break
    except:
        if attempt < 2:
            print(f"⏳ Intento {attempt + 1}/3 - Esperando servidor...")
            time.sleep(5)
        else:
            print("⚠️ El servidor tardó más de lo esperado, pero continuaremos...")

# Obtener URL pública de ngrok (con reintentos)
public_url = None
for attempt in range(5):
    try:
        response = requests.get('http://localhost:4040/api/tunnels', timeout=5)
        tunnels = response.json()['tunnels']
        if tunnels:
            public_url = tunnels[0]['public_url']
            break
    except Exception as e:
        if attempt < 4:
            print(f"⏳ Esperando ngrok API (intento {attempt + 1}/5)...")
            time.sleep(3)

if public_url:
    print("\n" + "="*70)
    print("✅ ✅ ✅  SERVIDOR LISTO  ✅ ✅ ✅")
    print("="*70)
    print(f"\n🌍 URL Pública: {public_url}")
    print("\n📋 Instrucciones:")
    print("   1. Haz click en el botón de abajo o copia la URL")
    print("   2. ¡Comienza a usar SimuPaciente!")
    print("\n⚠️  IMPORTANTE: NO detengas esta celda mientras uses la app")
    print("\n🔒 Seguridad: Esta URL es temporal y solo funciona mientras esta celda esté corriendo")
    print("="*70)
    
    # Mostrar enlace clickable
    display(HTML(f'''
        <div style="
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            padding: 30px;
            border-radius: 15px;
            margin: 20px 0;
            text-align: center;
            box-shadow: 0 10px 30px rgba(0,0,0,0.3);
        ">
            <h2 style="color: white; margin-bottom: 20px; font-size: 24px;">🏥 SimuPaciente UMH está listo</h2>
            <a href="{public_url}" target="_blank" style="
                display: inline-block;
                background: white;
                color: #667eea;
                padding: 15px 40px;
                text-decoration: none;
                border-radius: 25px;
                font-weight: bold;
                font-size: 18px;
                box-shadow: 0 5px 15px rgba(0,0,0,0.2);
                transition: transform 0.2s;
            " onmouseover="this.style.transform='scale(1.05)'" onmouseout="this.style.transform='scale(1)'">
                🚀 Abrir Aplicación
            </a>
            <p style="color: white; margin-top: 20px; font-size: 14px; opacity: 0.9;">
                URL: {public_url}
            </p>
        </div>
    '''))
else:
    print("\n❌ No se pudo obtener la URL de ngrok")
    print("\nPor favor:")
    print("   1. Detén esta celda (botón Stop)")
    print("   2. Ejecuta la celda nuevamente")
    print("\nSi el problema persiste, verifica tu conexión a Internet.")

# Mantener el servidor corriendo y mostrar logs
print("\n📊 Logs del servidor (CTRL+C para detener):")
print("-" * 70)

try:
    for line in server_process.stdout:
        print(line.rstrip())
except KeyboardInterrupt:
    print("\n🛑 Deteniendo servidor...")
    server_process.terminate()
    ngrok_process.terminate()
    print("✅ Servidor detenido")

---

## 📝 Notas Adicionales

### 🔄 Reiniciar el Servidor
Si necesitas reiniciar:
1. Detén la celda anterior (botón Stop)
2. Ejecuta la celda nuevamente

### 🐛 Solución de Problemas

**Problema:** No se conecta a OpenAI Realtime API
- Verifica que tu API key tiene acceso a Realtime API (beta)
- Comprueba que tienes créditos disponibles

**Problema:** La URL de ngrok no funciona
- Espera 30 segundos y recarga la página
- Verifica que la celda del servidor sigue corriendo [*]

### 📧 Soporte
- GitHub: https://github.com/marcosbenghezala/ECOE
- Universidad Miguel Hernández de Elche

---

**Versión:** 0.9.5 (Release Candidate)  
**Última actualización:** 17 de diciembre de 2025